# Welcome to SImulation of SEctoral Pathways with Uncertainty Exploration for DEcarbonization (SISEPUEDE)


https://sisepuede.readthedocs.io/en/latest/



In [2]:
## load packages
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import pathlib
import sys
import warnings
warnings.filterwarnings("ignore") # note: you can turn this on if you want to see warnings

# add SISEPUEDE to path
dir_py = pathlib.Path(os.path.realpath(".")).parents[0].joinpath("python")
if str(dir_py) not in sys.path:
    sys.path.append(str(dir_py))

# import some key elements
import model_attributes as ma
import sisepuede_file_structure as sfs
    


# First thing's first: let's access the file structure
The `sisepuede_file_structure.SISEPUEDEFileStructure` object stores relevant paths and the general file structure of the entire system It's a key piece of the `SISEPUEDE` object; however, we can use it to look at the most important object in the ecosystem, the `ModelAttributes` object.

In [3]:
# initialize a file structure
file_struct = sfs.SISEPUEDEFileStructure()

MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


# SISEPUEDE's entire model variable and unit ecosystem is managed by attribute tables
SISEPUEDE is driven primarily by a collection of CSV files located in the path stored at `file_struct.dir_attribute_tables`. This directory contains:
- Sector and Subsector attribute tables
- Category attribute tables
- Variable attribute/definition tables for each subsector
- Unit attribute/definition tables
- Other attribute tables used throughout the analysis

##  The `model_attributes.ModelAttributes` object organizes these

The `model_attributes.ModelAttributes` stores model variables (as `model_variable.ModelVariable` objects, unit conversion objects, methods for converting units between variables, methods for extracting variables from dataframes, methods for writing variables to data frames, scenario dimension management, and more. 

In [4]:
# here's where attribute tables are stored
file_struct.dir_attribute_tables

'/Users/jsyme/Documents/Projects/git_jbus/sisepuede/docs/source/csvs'

In [5]:
matt = file_struct.model_attributes

# Let's look at some of the functionality of the `ModelAttributes` object

First, let's look at the fundamental building blocks of data within the SISEPUEDE architecture: attribute tables. We can start with the *sector* and *subsector* attribute tables.
- Attribute tables are stored in the `attribute_table.AttributeTable` object, which includes some methods for mapping keys, dictionaries, etc.
    - The underlying dataframe is referenced as `AttributeTable.table`
- Showing it in a jupyter notebook displays the underlying dataframe

The **Sector** attribute table stores basic information about the 5 modeling sectors, including the shorthand id, the full name, and abbreviation, and a brief description. 

In [13]:
attr_sector = matt.get_sector_attribute_table()
print(type(attr_sector))
attr_sector



<class 'attribute_table.AttributeTable'>


,sector,sector_name,abbreviation_sector,description
0,AFOLU,"Agriculture, Forestry, and Land Use",af,Agricultural activity ....
1,Circular Economy,Circular Economy,ce,Activity (demand and emission) associated with...
2,Energy,Energy,en,Activity (demand and emission) associated with...
3,IPPU,Industrial Processes and Product Use,ip,Activity (demand and emission) associated with...
4,Socioeconomic,Socioeconomic,se,Socioeconomic and demographic activity affecti...


The **Subsector** attribute table stores basic information about the 21 modeling subsectors, including the shorthand id, the full name, and abbreviation, the **primary category**, a description, whether or not it is an emission subsector, and more. 
- The **Primary Category** is a variable schema element marker used to tell variables which category space they are defined within.
- As with other variable schema elements, it is wrapped in two hash marks (\`\`)

In [12]:
attr_subsector = matt.get_subsector_attribute_table()
attr_subsector.table.head()


,sector,subsector,abbreviation_subsector,primary_category,description,model_language,emission_subsector,primary_category_py
0,AFOLU,Agriculture,agrc,``$CAT-AGRICULTURE$``,Agricultural activity ....,Python,1,agriculture
1,AFOLU,Forest,frst,``$CAT-FOREST$``,"Forest activity, including sequestration and",Python,1,forest
2,AFOLU,Land Use,lndu,``$CAT-LANDUSE$``,"Land use activity, including deforestation and...",Python,1,landuse
3,AFOLU,Livestock Manure Management,lsmm,``$CAT-MANURE-MANAGEMENT$``,Pathways for treatment and use of manure. Incl...,Python,1,manure_management
4,AFOLU,Livestock,lvst,``$CAT-LIVESTOCK$``,Livestock activity and emissions.,Python,1,livestock


##  Each subsector is assocaited with 2 attribute tables

1. **Variable Definition** Attribute table, which contains definitions of all variables within that section
1. **Primary Category** Attribut table. The same primary category table can be associated with multiple subsectors (e.g., IPPU and Industrial Energy share the ``$CAT-INDUSTRY$`` category attribute table)




###  The **Variable Definition** attribute table defines all variables within a subsector

- Variables are defined in the table as a *schema*
    - Includes any primary categories, units, and any gasses
- Note: subsector names are stored as attributes in ``ModelAttributes`` as ``ModelAttributes.subsec_name_ABBREVIATION``, e.g., ``ModelAttributes.subsec_name_lndu`` for land use

###  Use the ``ModelAttributes.get_attribute_table`` method to retrieve attribute tables
- By default, the ``ModelAttributes.get_attribute_table`` method returns the category attribute

In [16]:
attr_lndu = matt.get_attribute_table(matt.subsec_name_lndu)
attr_lndu

,category_name,cat_landuse,definition,data_source,hyperlink,notes,cat_forest,crop_category,other_category,pasture_category,settlements_category,wetlands_category,mineralization_in_land_use_conversion_to_managed,reallocation_transition_probability_exhaustion_category
0,Croplands,croplands,Area of land devoted to growing crops for cons...,NaN,NaN,NaN,none,1,0,0,0,0,0,0
1,Forests - Mangroves,forests_mangroves,Area with mangroves,NaN,NaN,NaN,``mangroves``,0,0,0,0,0,1,0
2,Forests - Primary,forests_primary,"Area of land covered in primary forest (e.g., ...",NaN,NaN,NaN,``primary``,0,0,0,0,0,1,1
3,Forests - Secondary,forests_secondary,"Area of land covered in secondary forest, incl...",NaN,NaN,NaN,``secondary``,0,0,0,0,0,1,0
4,Grasslands,grasslands,"Area of grasslands, including native and pastu...",NaN,NaN,NaN,none,0,0,1,0,0,1,0
5,Other,other,"Other land use categories (including savannah,...",NaN,NaN,NaN,none,0,1,0,0,0,0,0
6,Settlements,settlements,Area of land devoted to urban/suburban develop...,NaN,NaN,NaN,none,0,0,0,1,0,0,0
7,Wetlands,wetlands,"Wetlands, which emit :math:`\text{CH}_4`",NaN,NaN,NaN,none,0,0,0,0,1,0,0


###  Use the ``ModelAttributes.get_attribute_table`` method to retrieve attribute tables
- In ``ModelAttributes.get_attribute_table``, set ``table_type = "variable_definitions"``

In [20]:
# to return the variable definiton, set 
attr_lndu = matt.get_attribute_table(matt.subsec_name_lndu, table_type = "variable_definitions")
attr_lndu

,variable_type,variable,information,variable_schema,categories,reference,default_value,default_lhs_scalar_minimum_at_final_time_period,default_lhs_scalar_maximum_at_final_time_period,simplex_group,emissions_total_by_gas_component,cat_soil_management,notes
0,Input,:math:\text{CO}_2 Land Use Conversion Emission...,Annual :math:`\text{CO}_2` emission factor for...,``ef_lndu_conv_$CAT-LANDUSE-DIM1$_to_$CAT-LAND...,all,NaN,0,1.0,1.0,NaN,0,none,NaN
1,Input,Fraction of Increasing Net Exports Met,| Fractional of changes to net exports to mee...,``frac_lndu_increasing_net_exports_met_$CAT-LA...,``croplands``|``grasslands``,NaN,0,1.0,1.0,NaN,0,none,NaN
2,Input,Fraction of Increasing Net Imports Met,| Fractional of changes to net imports to mee...,``frac_lndu_increasing_net_imports_met_$CAT-LA...,``croplands``|``grasslands``,NaN,0,1.0,1.0,NaN,0,none,NaN
3,Input,Fraction of Pastures Improved,Fraction of pasture that is improved using sus...,``frac_lndu_improved_$CAT-LANDUSE$``,``grasslands``,NaN,0,1.0,1.0,NaN,0,none,NaN
4,Input,Fraction of Soils Mineral,See ` Giulia Conchedda and Francesco N. Tubiel...,``frac_lndu_soil_mineral_$CAT-LANDUSE$``,``croplands``|``grasslands``|``forests_mangrov...,NaN,0,1.0,1.0,NaN,0,none,NaN
5,Input,Initial Land Use Area Proportion,Proportion of total **country land** area (%/1...,``frac_lndu_initial_$CAT-LANDUSE$``,all,NaN,0,1.0,1.0,1.0,0,none,NaN
6,Input,Land Use BOC :math:\text{CH}_4 Emission Factor,Annual average :math:`\text{CH}_4` emitted per...,``ef_lndu_boc_$CAT-LANDUSE$_$UNIT-MASS$_$EMISS...,``wetlands``,NaN,0,1.0,1.0,NaN,0,none,NaN
7,Input,Land Use Fraction Dry,| Used to calculate :math:`\text{N}_2\text{O}...,``frac_lndu_$CAT-LANDUSE$_cl2_dry``,``grasslands``|``other``|``settlements``,NaN,0,1.0,1.0,3.0,0,``dry_climate``,NaN
8,Input,Land Use Fraction Fertilized,Fraction of the land use category that receive...,``frac_lndu_receiving_fertilizer_$CAT-LANDUSE$``,``croplands``|``grasslands``,NaN,0,1.0,1.0,NaN,0,none,NaN
9,Input,Land Use Fraction Temperate,| Used to calculate :math:`\text{N}_2\text{O}...,``frac_lndu_$CAT-LANDUSE$_cl1_temperate``,``grasslands``,NaN,0,1.0,1.0,4.0,0,``temperate_crop_grass``,NaN


##  Now, let's grab a variable and look at it

SISEPUEDE distinguishes between *Variables* and *Variable Fields*.
- **Variables** are abstract objects that are associated with some number $\geq 0$ of categories within a subsector. All categories within the variable face the same underlying mathematical model when they are called in the variable system; however, difference categories may be associated with different values.
- **Variable Fields** are fields associated with a variable. These fields are defined in the relevant sectoral variable definition attribute table.

In [21]:
modvar = matt.get_variable("Area of Land Use Converted to Type")

In [23]:
modvar.fields

['area_lndu_conversion_to_croplands',
 'area_lndu_conversion_to_forests_mangroves',
 'area_lndu_conversion_to_forests_primary',
 'area_lndu_conversion_to_forests_secondary',
 'area_lndu_conversion_to_grasslands',
 'area_lndu_conversion_to_other',
 'area_lndu_conversion_to_settlements',
 'area_lndu_conversion_to_wetlands']

In [24]:
modvar.schema

In [44]:

modvar.dict_category_keys

{'cat_landuse': ['croplands',
  'forests_mangroves',
  'forests_primary',
  'forests_secondary',
  'grasslands',
  'other',
  'settlements',
  'wetlands']}